In [ ]:
# !pip3 install Faker
# pip install presidio_anonymizer
# !pip install presidio_analyzer presidio_anonymizer
# https://microsoft.github.io/presidio/analyzer/languages/#set-up-language-specific-recognizers

In [1]:
from presidio_analyzer import AnalyzerEngine, PatternRecognizer,EntityRecognizer,RecognizerRegistry
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig
from presidio_analyzer.nlp_engine import NlpEngineProvider
from presidio_anonymizer.entities import OperatorConfig, RecognizerResult
from faker import Faker
import pandas as pd
import json
import re
import spacy
import Levenshtein

2024-11-02 18:17:11.238946: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-02 18:17:11.240437: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-02 18:17:11.267864: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-02 18:17:11.863519: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# !pip3 install spacy-transformers

In [42]:
text_to_anonymize="""07.JAN.2012 16:42 0386284951 CLINIQUE CLAUDE BERNARD #5103P.001/003 Cacilo ADEMO Dr Annais BRILLIN-BORIS D OLANPOTOC-POMI Dr Sandra AGALI Pathologie Dr Brunetti PELLET wParis. Selaride medecins pathologistes conventionnes 89 bd de Stalin 927A0 MAT.AKOYF 559455880158652233 DR FETRET Christina 66RUEBUTTANT 75007PARIS Pxaman concexaanti MME gONZALEZ Gabriclle NomJF: Nei:01/11/195469as ossierT160285694 du20/11/2011 Noaéjour: opifutns CLINIQUE CLAUDE BERNARD (PMSI) COPIETELEMATIQUE D'UN COMPTE-RENDU J/TUMORECTOMIE SEIN.GAUCHEAYECEXAMENEXTEMPORANE III/IMMUNOHISTOCHIMIE 1/Un ganglion centimetrique est xepére par la sonde isotopique: les appositions per opératoires etPinclusion en paraffine fnontrent qu'il est indemne ca histologie conventionnelle. I1/La tumorectomie mesurant 5 x 3 x 1,5 cm est communiquée orientéc. Reponse extemporanée : foyer tumoral legerement excentre en dedans. Margee mncroacopiquct superieures A 5 mm. Apras inclusion en paraffine, le foyer mesure tur lame mm et comespond a une prolif&ration dc callules cubo cylindtiques a contouts imprecis a cytoplasme peu abondant.Les aoyaux claits ou hyperchrotnatiques pxésentent des variations de diamarre de 1 a 3, un cornpte mitotiquc d'une raitose pour dix champa HPF. Les cellules soat disposécs en travecs d'épaisseut variable au sein d'un stroxa fibreux. I cxiste une composante minoxitaire de carcinome canalaire in situ d'architecture campacte (gradc uterimédiaite saxs uéarose) representant 30% de la susface tumorale. Abseace d'embole vasculite a distance. CONCLUSION Carcinome canalaire infiltrant de 9 mm de grand axe aans cmbole vasculairo. Composante ininorivaire de carcinome canalaire in sinu (grade intermcdiair darchitecture Fompact) repréeentanr 30%de la surface tumorale. Marge microscopique saine la plus étroite de 5 mm en dedana. 3010 Un ganglion sendinclc indemne en hietologic conventionnelle. $BR II fort (3+3+1) index mitouique faible. No/1. Iaaunohistochimie cn coura. L01/03/2012 0386695550 07.JAN.201216:15 RECUDE: 3356-052"""

In [43]:
def createAnalyser():
    models = [{"lang_code": "fr", "model_name": "fr_core_news_lg"}]
    configuration = {"nlp_engine_name": "spacy", "models": models}
    provider = NlpEngineProvider(nlp_configuration=configuration)
    nlp_engine = provider.create_engine()
    analyzer = AnalyzerEngine(
                nlp_engine=nlp_engine, supported_languages=["fr"]
    )
    return analyzer

In [58]:
# class to recognize address numbers 
from presidio_analyzer import Pattern
from typing import Optional
from typing import List
# https://github.com/microsoft/presidio/blob/main/presidio-analyzer/presidio_analyzer/predefined_recognizers/email_recognizer.py
class AddressRecognizer(PatternRecognizer):
    """
    Recognize address nb and route using regex.

    :param patterns: List of patterns to be used by this recognizer
    :param context: List of context words to increase confidence in detection
    :param supported_language: Language this recognizer supports
    :param supported_entity: The entity this recognizer can detect
    """
    PATTERNS = [
        Pattern(
            "addresses medium",
            r"\b(\d{1,5})\s?(bis|ter|quater|a|b)?", # /b limite de mot
            0.1,
        ),
    ]

    # https://microsoft.github.io/presidio/tutorial/06_context/
    # So this is working, but would catch any 5 digit string. This is why we set the score to 0.01. Let's use context words to increase score:
    CONTEXT =  ["rue",
                "avenue",
                "boulevard",
                "place",
                "chemin",
                "route",
                "impasse",
                "allée",
                "passage", 
                "quai", 
                "bd" ]

    def __init__(
        self,
        patterns: Optional[List[Pattern]] = None,
        context: Optional[List[str]] = None,
        supported_language: str = "fr",
        supported_entity: str = "ADDRESS_NEW",
    ):
        patterns = patterns if patterns else self.PATTERNS
        context = context if context else self.CONTEXT
        super().__init__(
            supported_entity=supported_entity,
            patterns=patterns,
            context=context,
            supported_language=supported_language,
        )

class DoctorRecognizer(PatternRecognizer):
    """
    Recognize address nb and route using regex.

    :param patterns: List of patterns to be used by this recognizer
    :param context: List of context words to increase confidence in detection
    :param supported_language: Language this recognizer supports
    :param supported_entity: The entity this recognizer can detect
    """
    PATTERNS = [
        Pattern(
            "doc medium",
            r"[A-Z][a-z]+,?\s+(?:[A-Z][a-z]*\.?\s*)?[A-Z][a-z]+", # /b limite de mot  
            0.1,
        ),
    ]

    CONTEXT =  [
        "Dr.",
        "Dr",
        "Docteur",
        "Docteurs",
        "Profeseur",
        "dr",
        "prof.",
        "prof",
    ]

    def __init__(
        self,
        patterns: Optional[List[Pattern]] = None,
        context: Optional[List[str]] = None,
        supported_language: str = "fr",
        supported_entity: str = "DOCTOR_NAME",
    ):
        patterns = patterns if patterns else self.PATTERNS
        context = context if context else self.CONTEXT
        super().__init__(
            supported_entity=supported_entity,
            patterns=patterns,
            context=context,
            supported_language=supported_language,
        )

In [59]:
# ADDING reco
# https://microsoft.github.io/presidio/analyzer/adding_recognizers/#simple-example
def add_rulest(analyzer):
    titles_list = [
        "Madame",
        "Dr.",
        "Dr",
        "M.",
        "Mme.",
        "Mme",
        "Docteur",
        "Docteurs",
        "Profeseur",
        "mme.",
        "mme",
        "mlle.",
        "monsieur",
        "madame",
        "mademoiselle",
        "mr",
        "mrs",
        "dr",
        "mlle",
        "prof.",
        "prof",
        "patient",
        "patiente"
    ]
    
    titles_recognizer = PatternRecognizer(supported_entity="TITLE", deny_list=titles_list, supported_language="fr")
    # result = titles_recognizer.analyze(text_to_anonymize, entities=["TITLE"])
    # print(f"Result:\n {result}")
    # tt=EntityRecognizer(supported_entities="TITLE", name="ee", supported_language="fr",deny_list=titles_list)
    
    analyzer.registry.add_recognizer(titles_recognizer)
    analyzer.registry.add_recognizer(DoctorRecognizer())
    analyzer.registry.add_recognizer(AddressRecognizer())

    return analyzer
    


In [62]:
# Analyzer output
analyzer=createAnalyser()
analyzer=add_rulest(analyzer)
analyzer_results = analyzer.analyze(text=text_to_anonymize, language='fr', score_threshold=0.4)
print(f"Result:\n")
for e in analyzer_results: 
    print (e,":", text_to_anonymize[e.start:e.end])

anonymizer = AnonymizerEngine()
anonymized_results = anonymizer.anonymize(
    text=text_to_anonymize,
    analyzer_results=analyzer_results,    
    operators={"DEFAULT": OperatorConfig("replace", {"new_value": "<ANONYMIZED>"}), 
               "PERSON": OperatorConfig("replace", {"new_value": "<PERSON>"}), 
               "LOCATION": OperatorConfig("replace", {"new_value": "<LOCATION>"}), 
               "DATE_TIME": OperatorConfig("replace", {"new_value": "<DATE_TIME>"}),
                "PHONE_NUMBER": OperatorConfig("mask", {"type": "mask", "masking_char" : "*", "chars_to_mask" : 12, "from_end" : True}),
                 "TITLE": OperatorConfig("redact", {})}
)

print(f"text: {anonymized_results.text}")
# print("detailed response:")
# print(json.loads(anonymized_results.to_json()))

Result:

type: TITLE, start: 81, end: 83, score: 1.0 : Dr
type: TITLE, start: 122, end: 124, score: 1.0 : Dr
type: TITLE, start: 149, end: 151, score: 1.0 : Dr
type: TITLE, start: 273, end: 275, score: 1.0 : DR
type: TITLE, start: 336, end: 339, score: 1.0 : MME
type: PERSON, start: 81, end: 90, score: 0.85 : Dr Annais
type: PERSON, start: 125, end: 137, score: 0.85 : Sandra AGALI
type: LOCATION, start: 168, end: 174, score: 0.85 : wParis
type: LOCATION, start: 176, end: 184, score: 0.85 : Selaride
type: PERSON, start: 231, end: 237, score: 0.85 : Stalin
type: PERSON, start: 283, end: 292, score: 0.85 : Christina
type: LOCATION, start: 415, end: 423, score: 0.85 : Noaéjour
type: PERSON, start: 425, end: 449, score: 0.85 : opifutns CLINIQUE CLAUDE
type: PERSON, start: 515, end: 551, score: 0.85 : SEIN.GAUCHEAYECEXAMENEXTEMPORANE III
type: LOCATION, start: 828, end: 840, score: 0.85 : extemporanée
type: PERSON, start: 888, end: 894, score: 0.85 : Margee
type: LOCATION, start: 1229, end: 

In [63]:
fake = Faker(locale=['fr_FR'])
for i in range(5):
    print(fake.name())
    # print(fake.date('%d/%m/%Y'))
    # print(fake.address())
          


Benoît de la Hubert
Eugène Chauvin
Bertrand Boucher
Isabelle de la Leduc
Anouk Hardy-Fleury


In [64]:
# Create faker function (note that it has to receive a value)
dictionary_anonymized_people={}
last_name="Gonzalez"
# TODO: when anonimyse person: keep the same fake name
def fake_person(input_text):
    if input_text in dictionary_anonymized_people:
        return dictionary_anonymized_people[input_text]
    elif input_text.find(last_name)>-1:
        new_name=fake.name_female()
        dictionary_anonymized_people[input_text]=new_name
        return new_name
    else:
        new_name=fake.name()
        dictionary_anonymized_people[input_text]=new_name
        return new_name


def fake_location(input_text):
    """
    Parameters:
        input_text (str): The input text from which to generate the fake location.
    Returns:
        str: The generated fake location name or address.
    """
    fake = Faker("fr_FR")
    address_keywords =  ["rue",
                "avenue",
                "boulevard",
                "place",
                "chemin",
                "route",
                "impasse",
                "allée",
                "passage", 
                "quai", 
                "bd" ]
    has_comma = "," in input_text
    if has_comma:
        # Assume it's an address with comma-separated elements
        return " {} ".format(fake.address())

    input_lower = input_text.lower()

    if any(keyword in input_lower for keyword in address_keywords):
        # Generate a fake full address
        return " {} ".format(fake.address())
    # Generate a fake Swiss city name
    fake_city = fake.city()
    # add_name(input_text, "CITY", fake_name)
    return " {} ".format(fake_city)


In [65]:
anonymizer = AnonymizerEngine()
anonymized_text = anonymizer.anonymize(text=text_to_anonymize,
            analyzer_results=analyzer_results,
                                       # operators=fake_operators
                                       # ).text
            operators={"DEFAULT": OperatorConfig("replace", {"new_value": "<ANONYMIZED>"}), 
            # "ADDRESS_NUMBER": OperatorConfig("custom", {"lambda": lambda x: fake.address()}),
            "ADDRESS_NUMBER": OperatorConfig("replace", {"new_value": "<ADDRESS_NUMBER>"}), 
            # "PERSON": OperatorConfig("custom", {"lambda": lambda x: fake.name()}),
            "PERSON": OperatorConfig("custom", {"lambda": fake_person}),
            # "LOCATION": OperatorConfig("replace", {"new_value": "<LOCATION>"}), 
            "LOCATION": OperatorConfig("custom", {"lambda": fake_location}), 
            "DATE_TIME": OperatorConfig("custom",{"lambda": lambda x: fake.date()}),
           "EMAIL_ADDRESS": OperatorConfig("custom", {"lambda": lambda x: fake.email()}),
           "PHONE_NUMBER": OperatorConfig("custom", {"lambda": lambda x: fake.phone_number()}),
            "TITLE": OperatorConfig("redact", {})}
)
print(f"text: {anonymized_text}")

text: text: 07.JAN.2012 16:42 0778075002 CLINIQUE CLAUDE BERNARD #5103P.001/003 Cacilo ADEMO Antoinette Valentin<ANONYMIZED>-<ANONYMIZED>-POMI  Antoinette Hamel<ANONYMIZED>. <ANONYMIZED> conventionnes 89 bd de André Dos Santos <ANONYMIZED>0 MAT.AKOYF <ANONYMIZED>5880158652233 <ANONYMIZED> <ANONYMIZED>RUEBUTTANT <ANONYMIZED><ANONYMIZED>  gONZALEZ Gabriclle NomJF: Nei:01/11/195469as ossierT160285694 du20/11/2011  Petit-les-Bains : Victor Blin-Laporte BERNARD (PMSI) COPIETELEMATIQUE D'UN COMPTE-RENDU J/TUMORECTOMIE Jacqueline Lévêque/IMMUNOHISTOCHIMIE 1/Un ganglion centimetrique est xepére par la sonde isotopique: les appositions per opératoires etPinclusion en paraffine fnontrent qu'il est indemne ca histologie conventionnelle. I1/La tumorectomie mesurant 5 x 3 x 1,5 cm est communiquée orientéc. Reponse  Lemaître  : foyer tumoral legerement excentre en dedans. Dorothée Rodrigues mncroacopiquct superieures A 5 mm. Apras inclusion en paraffine, le foyer mesure tur lame mm et comespond a un

In [74]:
df_docFile=pd.read_json('../data/data_jsons/docFile_paddleOcr.json')
# load the dictionary with patienst data 
# get patient data to replace in text!
df_pat_names=pd.read_json('../data/data_jsons/dict_pname_par_diag.json')
df_doc_names=pd.read_json('../data/data_jsons/dict_diag_par.json')

def remove_extra_spaces(txt):
    # remove extra spaces
    txt = " ".join(txt.split())
    txt = txt.replace(" ,", ",").replace(" .", ".")
    # txt = " ".join(txt.split())
    # remove extra simbols
    txt = re.sub(r"([.,])\1+", r"\1", txt)
    return txt

fake_operators={"DEFAULT": OperatorConfig("replace", {"new_value": "<ANONYMIZED>"}), 
# "ADDRESS_NUMBER": OperatorConfig("custom", {"lambda": lambda x: fake.address()}),
"ADDRESS_NEW": OperatorConfig("replace", {"new_value": "<ADDRESS_NEW>"}), 
# "PERSON": OperatorConfig("custom", {"lambda": lambda x: fake.name()}),
"PERSON": OperatorConfig("custom", {"lambda": fake_person}),
# "LOCATION": OperatorConfig("replace", {"new_value": "<LOCATION>"}), 
"LOCATION": OperatorConfig("custom", {"lambda": fake_location}), 
"DATE_TIME": OperatorConfig("custom",{"lambda": lambda x: fake.date()}),
"EMAIL_ADDRESS": OperatorConfig("custom", {"lambda": lambda x: fake.email()}),
"PHONE_NUMBER": OperatorConfig("custom", {"lambda": lambda x: fake.phone_number()}),
"TITLE": OperatorConfig("replace", {"new_value": "<TITLE>"})
               }

text_to_anonymize="""07.JAN.2012 16:42 0386284951 CLINIQUE CLAUDE BERNARD #5103P.001/003 Cacilo ADEMO Dr Annais BRILLIN-BORIS D OLANPOTOC-POMI Dr Sandra AGALI Pathologie Dr Brunetti PELLET wParis. Selaride medecins pathologistes conventionnes 89 bd de Stalin 927A0 MAT.AKOYF 559455880158652233 DR FETRET Christina 66RUEBUTTANT 75007PARIS Pxaman concexaanti MME gONZALEZ Gabriclle NomJF: Nei:01/11/195469as ossierT160285694 du20/11/2011 Noaéjour: opifutns CLINIQUE CLAUDE BERNARD (PMSI) COPIETELEMATIQUE D'UN COMPTE-RENDU J/TUMORECTOMIE SEIN.GAUCHEAYECEXAMENEXTEMPORANE III/IMMUNOHISTOCHIMIE 1/Un ganglion centimetrique est xepére par la sonde isotopique: les appositions per opératoires etPinclusion en paraffine fnontrent qu'il est indemne ca histologie conventionnelle. I1/La tumorectomie mesurant 5 x 3 x 1,5 cm est communiquée orientéc. Reponse extemporanée : foyer tumoral legerement excentre en dedans. Margee mncroacopiquct superieures A 5 mm. Apras inclusion en paraffine, le foyer mesure tur lame mm et comespond a une prolif&ration dc callules cubo cylindtiques a contouts imprecis a cytoplasme peu abondant.Les aoyaux claits ou hyperchrotnatiques pxésentent des variations de diamarre de 1 a 3, un cornpte mitotiquc d'une raitose pour dix champa HPF. Les cellules soat disposécs en travecs d'épaisseut variable au sein d'un stroxa fibreux. I cxiste une composante minoxitaire de carcinome canalaire in situ d'architecture campacte (gradc uterimédiaite saxs uéarose) representant 30% de la susface tumorale. Abseace d'embole vasculite a distance. CONCLUSION Carcinome canalaire infiltrant de 9 mm de grand axe aans cmbole vasculairo. Composante ininorivaire de carcinome canalaire in sinu (grade intermcdiair darchitecture Fompact) repréeentanr 30%de la surface tumorale. Marge microscopique saine la plus étroite de 5 mm en dedana. 3010 Un ganglion sendinclc indemne en hietologic conventionnelle. $BR II fort (3+3+1) index mitouique faible. No/1. Iaaunohistochimie cn coura. L01/03/2012 0386695550 07.JAN.201216:15 RECUDE: 3356-052"""

current_idx=0
for idx in [4]:#df_docFile.index: 
    current_idx=idx
    if idx%100==0:
        print(idx)
        
    # text_to_anonymize=df_docFile["contents"][idx].replace('\n', ' ')
    text_to_anonymize=remove_extra_spaces(text_to_anonymize)
    # last name
    # last_name=df_pat_names[df_docFile["id_diagnostique"][idx]]["nom"]
    last_name="GONZALEZ"
    first_name="Gabrielle" #df_pat_names[df_docFile["id_diagnostique"][idx]]["prenom"]
    nom_jeune="" #df_pat_names[df_docFile["id_diagnostique"][idx]]["nom_jeune"]
    analyzer_results = analyzer.analyze(text=text_to_anonymize, language='fr', score_threshold=0.4)
    # entities=["DATE_TIME","TITLE","ADDRESS_NUMBER","EMAIL_ADDRESS","LOCATION","PERSON","PHONE_NUMBER"],
    print(analyzer_results)
    print("--------------------------------------------------------------")
    anonymized_text = anonymizer.anonymize(text=text_to_anonymize,
                                           analyzer_results=analyzer_results,
                                           operators=fake_operators
                                           ).text
    
    lower_lname = str.lower(last_name)
    arr_lname=str.split(lower_lname," ")
    lower_fname = str.lower(first_name)
    arr_fname=str.split(lower_fname," ")
    lower_jfname= str.lower(nom_jeune)
    # date_naissance: "1958-07-30"
    date_nai= "01-11-1954"#str.split(df_pat_names[df_docFile["id_diagnostique"][idx]]["date_naissance"],"-")
    
    # Replacing all occurrences of substring s1 with s2
    anonymized_text = re.sub(r'(?i)'+last_name, fake.last_name(), anonymized_text)
    # first name
    if (len(arr_fname)>0):
        for search_text in arr_fname:
            # search_text=arr_fn[ind_firstname] #df_pat_names[df_docFile["id_diagnostique"][idx]]["prenom"]
            # Replacing all occurrences of substring s1 with s2
            anonymized_text = re.sub(r'(?i)'+search_text, fake.first_name(), anonymized_text)
    # single name
    # Replacing all occurrences of substring s1 with s2
    if lower_jfname!="":
        anonymized_text = re.sub(r'(?i)'+lower_jfname, fake.last_name(),anonymized_text)

    #birth date
    anonymized_text = re.sub(r'(?i)'+date_nai, "01/01/2001",anonymized_text)

    print(anonymized_text)
    # df_docFile.loc[idx,'contents']=anonymized_text
    print("--------------------------------------------------------------")
    # print(text_to_anonymize)


[type: TITLE, start: 81, end: 83, score: 1.0, type: TITLE, start: 122, end: 124, score: 1.0, type: TITLE, start: 149, end: 151, score: 1.0, type: TITLE, start: 273, end: 275, score: 1.0, type: TITLE, start: 336, end: 339, score: 1.0, type: PERSON, start: 81, end: 90, score: 0.85, type: PERSON, start: 125, end: 137, score: 0.85, type: LOCATION, start: 168, end: 174, score: 0.85, type: LOCATION, start: 176, end: 184, score: 0.85, type: PERSON, start: 231, end: 237, score: 0.85, type: PERSON, start: 283, end: 292, score: 0.85, type: LOCATION, start: 415, end: 423, score: 0.85, type: PERSON, start: 425, end: 449, score: 0.85, type: PERSON, start: 515, end: 551, score: 0.85, type: LOCATION, start: 828, end: 840, score: 0.85, type: PERSON, start: 888, end: 894, score: 0.85, type: LOCATION, start: 1229, end: 1235, score: 0.85, type: PERSON, start: 1254, end: 1268, score: 0.85, type: LOCATION, start: 1422, end: 1454, score: 0.85, type: PERSON, start: 1497, end: 1504, score: 0.85, type: PERSON,